In [1]:
import os
import sys
import pywt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
sys.path.append(os.path.abspath(os.path.join(r"../../Seismic-wave/")))

from utils.data_utils import split_3_channel, test_train_split

In [2]:
feature_path = "../data/processed/balance_16k/preprocessing/x_train.csv"
label_path = "../data/processed/balance_16k/preprocessing/y_train.csv"

df = pd.read_csv(feature_path)
df_label = pd.read_csv(label_path)

train, _ = test_train_split(df, df_label, split=0.001)

train_features, train_labels = split_3_channel(train)

Index(['S'], dtype='object')


In [3]:
def znorm(data):
    return (data - np.mean(data)) / (np.std(data)+ 0.001)


def min_max(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


def apply_cwt_to_all_rows(data, scales, waveletname):
    cwt_results = []

    for i in range(data.shape[0]):
        BHZ = data[i][0]
        BHN = data[i][1]
        BHE = data[i][2]
        BHZ = znorm(data[i][0])
        BHN = znorm(data[i][1])
        BHE = znorm(data[i][2])
        cwt_BHZ, _ = pywt.cwt(BHZ, scales, waveletname)
        cwt_BHN, _ = pywt.cwt(BHN, scales, waveletname)
        cwt_BHE, _ = pywt.cwt(BHE, scales, waveletname)

        cwt_combined = np.stack([cwt_BHZ, cwt_BHN, cwt_BHE], axis=0)
        cwt_results.append(cwt_combined)
        
    return np.array(cwt_results)

def save_spectrogram_images_from_cwt(cwt_data, labels, output_folder):
    num_samples = cwt_data.shape[0]
    num_scales = cwt_data.shape[2]
    o = output_folder

    for i in range(num_samples):
        output_folder = o
        sample_data = cwt_data[i]
        label = labels[i]

        BHZ = np.array(sample_data[0])
        BHN = np.array(sample_data[1])
        BHE = np.array(sample_data[2])
        composite_image = np.zeros((num_scales, BHZ.shape[1], 3))
        
        BHZ = znorm(BHZ)
        BHN = znorm(BHN)
        BHE = znorm(BHE)

        composite_image[:, :, 0] = np.abs(BHZ)
        composite_image[:, :, 1] = np.abs(BHN)
        composite_image[:, :, 2] = np.abs(BHE)

        # composite_image = ndimage.gaussian_filter(composite_image, sigma=0.0015)
        # composite_image  = composite_image * 0.3
        # composite_image  = np.abs(composite_image)
        composite_image = min_max(composite_image)

        if np.isnan(composite_image).any() or np.isinf(composite_image).any():
            print(
                f"Error: NaN or Inf values detected in composite_image for sample {i}. Skipping this sample."
            )
            continue

        # max_value = composite_image.max()
        # if max_value != 0:
        #     composite_image /= max_value
        # else:
        #     print(f"Warning: Max value is zero for sample {i}. Skipping normalization.")
        #     continue
        filename = f"{i}.png"
        if label == "Lg":
            output_folder += f"/{label}"
        elif label == "P":
            output_folder += f"/{label}"
        elif label == "Pg":
            output_folder += f"/{label}"
        elif label == "S":
            output_folder += f"/{label}"
        else:
            output_folder += f"/{label}"
        os.makedirs(output_folder, exist_ok=True)
        filepath = os.path.join(output_folder, filename)
        plt.figure(figsize=(10, 6))
        plt.imshow(composite_image, aspect="auto", cmap='jet')
        plt.axis("off")

        plt.savefig(filepath, format="png", bbox_inches="tight", pad_inches=0)
        plt.close()

In [4]:
waveletname = 'cmor1.5-1.0' 
# waveletname = 'mexh' 
scales = np.geomspace(1, 5024, num=200)
num_columns_per_channel = 2401
cwt_transformed_data = apply_cwt_to_all_rows(train_features,scales, waveletname)

output_folder = "../spectrogram_images/"
save_spectrogram_images_from_cwt(cwt_transformed_data, train_labels, output_folder)